In [58]:
# model 

from torch import nn
import torch.nn.functional as F
import numpy as np
import torch


### predefined functions and classes ###

def get_acc(adj_rec, adj_label):
    labels_all = adj_label.view(-1).long()
    preds_all = (adj_rec > 0.5).view(-1).long()
    accuracy = (preds_all == labels_all).sum().float() / labels_all.size(0)
    return accuracy

def glorot_init(input_dim, output_dim):
    init_range = np.sqrt(6.0/(input_dim + output_dim))
    initial = torch.rand(input_dim, output_dim)*2*init_range - init_range
    return nn.Parameter(initial)

def del_tensor_ele(arr, index):
    arr1 = arr[0:index]
    arr2 = arr[index+1:]
    return torch.cat((arr1, arr2), dim=0)

class GraphConvSparse(nn.Module):
    def __init__(self, input_dim, output_dim, activation = F.relu, **kwargs):
        super(GraphConvSparse, self).__init__(**kwargs)
        self.weight = glorot_init(input_dim, output_dim) 
        self.activation = activation
        self.input_dim = input_dim
        self.output_dim = output_dim

    def forward(self, adj, inputs):
        sample = torch.randn(1, adj.size(1), self.output_dim)
        for i in range(adj.size(0)):
            x_ = inputs[i].reshape(inputs.size(1), inputs.size(2))
            adj_ = adj[i].reshape(adj.size(1), adj.size(2))
            x_ = torch.mm(x_,self.weight.double())
            x_ = torch.mm(adj_, x_)
            outputs = self.activation(x_)
            outputs = outputs.reshape(1, adj.size(1), self.output_dim)
            sample = torch.cat((sample, outputs), dim=0)
        sample = del_tensor_ele(sample, 0)
        return sample

def dot_product_decode(Z):
    A_pred = torch.sigmoid(torch.matmul(Z,Z.t()))
    return A_pred

### VGAE class ###
class VGAE(nn.Module):
    def __init__(self):
        self.base_gcn = GraphConvSparse(input_dim, hidden1_dim)
        self.gcn_mean = GraphConvSparse(hidden1_dim, hidden2_dim, activation=lambda x:x)
        self.gcn_logstddev = GraphConvSparse(hidden1_dim, hidden2_dim, activation=lambda x:x)
  
    def encode(self, adj, X, n):
        hidden = self.base_gcn(adj, X)
        mean_ = self.gcn_mean(adj, hidden)
        logstd = self.gcn_logstddev(adj, hidden)
        var = torch.pow(torch.exp(logstd),2)
        self.mean = torch.mean(mean_, axis=0).reshape(1, adj.size(1), hidden2_dim)
        self.logstd = torch.log(torch.pow((torch.mean(var, axis=0)+torch.var(mean_, axis=0)),0.5)).reshape(1, adj.size(1), hidden2_dim)
        sampled_z = torch.randn(1, adj.size(1), hidden2_dim)
        for i in range(n):
            gaussian_noise = torch.randn(adj.size(1), hidden2_dim).reshape(1, adj.size(1), hidden2_dim)
            z = gaussian_noise*torch.exp(self.logstd) + self.mean
            sampled_z = torch.cat((sampled_z, z), dim=0)
        sampled_z = del_tensor_ele(sampled_z, 0)
        return sampled_z
    
    def forward(self, adj, X, n):
        Z = self.encode(adj, X, n)[0]
        z = Z.reshape(adj.size(1), hidden2_dim)
        A_pred = dot_product_decode(z).reshape(1, adj.size(1), adj.size(1))
        A_pred_ = A_pred
        for i in range(n-1):
            A_pred = torch.cat((A_pred, A_pred_), dim=0)
        return A_pred

In [24]:
from os import walk
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from scipy.sparse import csr_matrix


def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    return sparse_to_tuple(adj_normalized)

def norm_graph(adj):
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = np.diag(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt)
    return adj_normalized

def get_filename(mypath):
    f = []
    for (dirpath, dirnames, filenames) in walk(mypath):
        f.extend(filenames)
        break
    return f

def compute_cent(mat):
    return (np.sum(mat, axis=0))



class LoadData():
    """Load graph data"""
    def __init__(self, file_path, samples, non_orphan_list=None, n_node=38):
        self.samples = samples
        self.file_path = file_path
        self.nSample = len(samples)
        self.non_orphan_list = non_orphan_list
        self.n_node = n_node
        self.n_connect = len(non_orphan_list)

    def get_adj_wgh(self):
        """generating weighted adjacency matrix"""
        adj_orig_list =[]
        for sample in self.samples:
            f_name = self.file_path + "/"+ sample
            adj = np.asarray(pd.read_csv(f_name, index_col = 0, iterator = False))
            adj_orig_list.append(adj)    
        return adj_orig_list

    def get_adj_m(self):
        """generating the 0-1 adjacency matrix without diagonal elements"""
        adj_wgh = self.get_adj_wgh()
        adj_m_list =[]
        for _, adj in enumerate(adj_wgh):
            adj[adj > 0] = 1
            for i in range(self.n_node):
                adj[i, i] = 0
            for i in range(self.n_node):
                for j in range(self.n_node):
                    if adj[i, j] == 1:
                        adj[j, i] = 1
            for i in range(self.n_node - 1, -1, -1):
                if i not in self.non_orphan_list:
                    adj = np.delete(adj, (i), axis=0)
                    adj = np.delete(adj, (i), axis=1)
            adj_m_list.append(adj)    
        return adj_m_list
    
    def adj_without(self):
        adj_wgh = self.get_adj_wgh()
        sample = np.eye(self.n_connect)
        adj_without = np.array([sample])
        for _, adj in enumerate(adj_wgh):
            adj[adj > 0] = 1
            for i in range(self.n_node):
                adj[i, i] = 0
            for i in range(self.n_node):
                for j in range(self.n_node):
                    if adj[i, j] == 1:
                        adj[j, i] = 1
            for i in range(self.n_node - 1, -1, -1):
                if i not in self.non_orphan_list:
                    adj = np.delete(adj, (i), axis=0)
                    adj = np.delete(adj, (i), axis=1)
            adj = adj.astype(np.float)
            adj_without = np.append(adj_without, [adj], axis=0)
        adj_without = np.delete(adj_without, 0, axis=0)
        return adj_without

    def get_adj_m_t(self):
        """0-1 adjacency matrix without diagonal elements tensor format"""
        adj_wgh = self.get_adj_wgh()
        adj_m_list =[]
        for _, adj in enumerate(adj_wgh):
            adj[adj > 0] = 1
            for i in range(self.n_node):
                adj[i, i] = 0
            for i in range(self.n_node):
                for j in range(self.n_node):
                    if adj[i, j] == 1:
                        adj[j, i] = 1
            for i in range(self.n_node - 1, -1, -1):
                if i not in self.non_orphan_list:
                    adj = np.delete(adj, (i), axis=0)
                    adj = np.delete(adj, (i), axis=1)
            adj = sparse_to_tuple(sp.coo_matrix(adj))
            adj = torch.sparse.FloatTensor(torch.LongTensor(adj[0].T), 
                            torch.FloatTensor(adj[1]), 
                            torch.Size(adj[2]))
            adj_m_list.append(adj)    
        return adj_m_list
    
    def get_adj_label(self):
        """0-1 adjancency matrix with diagonal elements tensor format"""
        adj_m = self.get_adj_m()
        adj_label_list =[]
        
        for _, adj in enumerate(adj_m):
            adj_label = adj + sp.eye(adj.shape[0])
            adj_label = sparse_to_tuple(sp.coo_matrix(adj_label))
            #adj_label = sparse_to_tuple(adj_label)
            adj_label = torch.sparse.FloatTensor(torch.LongTensor(adj_label[0].T), 
                            torch.FloatTensor(adj_label[1]), 
                            torch.Size(adj_label[2]))
            adj_label_list.append(adj_label)
        return adj_label_list

    def adj_with(self):
        adj_m = self.get_adj_m()
        sample = np.eye(self.n_connect)
        adj_with = np.array([sample])
        for _, adj in enumerate(adj_m):
            adj_label = adj + sp.eye(adj.shape[0])
            adj_label = adj_label.astype(np.float)
            adj_with = np.append(adj_with, [adj_label], axis=0)
        adj_with = np.delete(adj_with, 0, axis=0)
        return adj_with

    def get_adj_norm(self):
        """0-1 normalized adjancency matrix tensor format"""
        adj_m = self.get_adj_m()
        adj_norm_list =[]

        for _, adj in enumerate(adj_m):
            adj_norm = preprocess_graph(adj)
            adj_norm = torch.sparse.FloatTensor(torch.LongTensor(adj_norm[0].T), 
                            torch.FloatTensor(adj_norm[1]), 
                            torch.Size(adj_norm[2]))
            adj_norm_list.append(adj_norm)
        return adj_norm_list

    def adj_norm(self):
        adj_m = self.get_adj_m()
        sample = np.eye(self.n_connect)
        adj_norm = np.array([sample])
        for _, adj in enumerate(adj_m):
            adj_norm_ = norm_graph(adj)
            adj_norm_ = adj_norm_.astype(np.float)
            adj_norm = np.append(adj_norm, [adj_norm_], axis=0)
        adj_norm = np.delete(adj_norm, 0, axis=0)
        return adj_norm

    def get_feature(self):
        """generating feature matrix X tensor format"""
        adj_wgh = self.get_adj_wgh()
        x_list = []
        for _, adj in enumerate(adj_wgh):
            x_feature  = adj
            x_feature  = csr_matrix(x_feature)
            x_feature  = sparse_to_tuple(x_feature)
            x_feature  = torch.sparse.FloatTensor(torch.LongTensor(x_feature[0].T), 
                            torch.FloatTensor(x_feature[1]), 
                            torch.Size(x_feature[2]))
            x_list.append(x_feature)
        return x_list

    def get_x_feature(self):
        """one-hot encoding tensor format"""
        x_onehot = torch.eye(len(self.non_orphan_list))
        return torch.FloatTensor(x_onehot)

    def one_hot(self):
        sample = np.eye(self.n_connect)
        feature = np.array([sample])
        for i in range(self.nSample):
            feature = np.append(feature,[np.eye(self.n_connect)],axis=0)
        feature = np.delete(feature, 0, axis=0)
        return feature







In [59]:

import torch
from torch.autograd.grad_mode import F
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
import scipy.sparse as sp
import numpy as np
import networkx as nx

class SimuData():
    """Simulate graph data"""
    def __init__(self, n_node=10, n_graph=30):
        self.n_node = n_node
        self.n_graph = n_graph

    def simu_adj_wgh(self):
        adj_list = []

        for i in range(self.n_graph):
            ## zero matrix
            A = torch.zeros(self.n_node, self.n_node)

            # first five nodes weights: uniform(0,1)A[:5,:5] = W
            W = torch.rand(5,5)

            ## symmetric
            i, j = torch.triu_indices(5, 5)
            W[i, j] = W.T[i, j]

            A[:5,:5] = W
            adj_list.append(A)  

        return adj_list

    def simu_adj_diag(self):
        adj_list = []

        for i in range(self.n_graph):
            A = torch.eye(self.n_node)
            adj_list.append(A)  

        return adj_list

    def simu_adj_m(self):
        """generating adjacency matrix"""
        adj_wgh = self.simu_adj_wgh()
        #adj_wgh = self.simu_adj_diag()
        adj_m_list =[]
        for _, adj in enumerate(adj_wgh):
            adj[adj>0.5] = 1
            adj[adj<0.5] = 0
            adj_m_list.append(adj)    
        return adj_m_list

    def graph_dataset(self):
        dataset =[]
        simu_adj = self.simu_adj_m()

        for _, adj in enumerate(simu_adj):
            edge_index_temp = sp.coo_matrix(adj)
            indices = np.vstack((edge_index_temp.row, edge_index_temp.col))
            edge_index_A = torch.LongTensor(indices) 
            x = self.get_x_feature()
            data = Data(x=x, edge_index=edge_index_A)
            dataset.append(data)

        return dataset

    def get_x_feature(self):
        x = torch.arange(self.n_node)
        x_onehot = torch.eye(self.n_node)[x,:] 

        return torch.FloatTensor(x_onehot)


class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) 
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)


def train():
    transform = T.Compose([
                T.NormalizeFeatures(),
                T.RandomLinkSplit(num_val=0, num_test=0, is_undirected=True,
                                split_labels=True, add_negative_train_samples=False),])
    
    simu_graph = SimuData()
    dataset = simu_graph.graph_dataset()
    n_node = simu_graph.n_node
    
    out_channels = 2
    num_features = n_node
    
    model = VGAE()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    epochs = 100

    model.train()
    
    for epoch in range(epochs):
        loss_total = 0
        optimizer.zero_grad()
        
        for i in range(len(dataset)):
            train_data, val_data, test_data = transform(dataset[i])
            z = model.encode(train_data.x, train_data.edge_index)
            loss = model.recon_loss(z, train_data.pos_edge_label_index)
            loss = loss + (1 /num_features) * model.kl_loss()
            loss_total += loss

        loss_avg = loss_total/len(dataset)
        loss_avg.backward()
        optimizer.step()
        
        print(loss_avg)

train()
    


AttributeError: cannot assign module before Module.__init__() call

In [52]:
import torch
from torch.utils.data import Dataset, TensorDataset
from torchvision import datasets, transforms
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision.models as models
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
from sklearn.metrics import roc_auc_score, average_precision_score
import time
import sys
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random
from scipy.stats import ttest_ind, levene
import scipy.stats as stats
from scipy.sparse import csr_matrix
from sklearn.mixture import GaussianMixture

### change working directory ###

    
### set parameters ###
ipf_sample_size = 32
control_sample_size = 28

num_node = 38
input_dim = 38
hidden1_dim = 8
hidden2_dim = 4
num_epoch = 100
learning_rate = 0.01


w1 = 1
w2 = 1
w = 1e-4
norm1 = 1
norm2 = 1e-4

###################################################################################
######### data analysis for all_collagens_ipf_data ##########
###################################################################################

path_3 = './ST-Aim3/vgae_data/result_all_raw/EGF/IPF'  ## all_egf_ipf ##
path_4 = './vgae_data/result_all_raw/EGF/Control'  ## all_egf_control ##
path_9 = './vgae_data/result_sig_raw/EGF/IPF'  ## sig_egf_ipf ##
path_10 = './vgae_data/result_sig_raw/EGF/Control'  ## sig_egf_control ##


### specific parameters ###

path = path_3
sample_size = ipf_sample_size
train_size = sample_size
test_size = 0

data_full = LoadData(samples=get_filename(path), file_path=path, non_orphan_list=range(38))
cen_full = []
cen = []

for i in range(sample_size):
    cen_full.append(compute_cent(data_full.get_adj_m()[i]))
non_orphan = np.argwhere(np.sum(cen_full, axis=0)!=0).flatten().tolist()
data = LoadData(file_path=path, samples=get_filename(path), non_orphan_list=non_orphan, n_node=num_node)
for i in range(sample_size):
    cen.append(compute_cent(data.get_adj_m()[i]))

### construct data loader ###
adj_without = torch.from_numpy(data.adj_without())
adj_with = torch.from_numpy(data.adj_with())
adj_norm = torch.from_numpy(data.adj_norm())
one_hot = torch.from_numpy(data.one_hot())
dataset = TensorDataset(adj_without, adj_with, adj_norm, one_hot)
loader = DataLoader(dataset = dataset,batch_size = train_size)


### train model for ipf data###

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate*(0.3**(epoch//100))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

model = VGAE(encoder=Encoder)
optimizer = Adam(model.parameters(), lr=learning_rate)

loss_value = []
acc_value = []
lik_value = []

for epoch in range(num_epoch):
    t = time.time()
    adjust_learning_rate(optimizer, epoch)
    model.train()
    for adj_without, adj_with, adj_norm, features in loader:

        A_pred = model(adj_norm, features, sample_size)
        mean = model.mean.reshape(adj_with.size(1), hidden2_dim)
        logstd = model.logstd.reshape(adj_with.size(1), hidden2_dim)
        loss = loss_function(w1,w2,w,norm1,norm2,A_pred,mean,logstd,adj_without,adj_with)
        train_acc = get_acc(A_pred,adj_with)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_value.append(loss.item())
        acc_value.append(train_acc.item())

    if epoch % 5 ==0:
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(loss.item()), 
              "train_acc=", "{:.5f}".format(train_acc),"time=", "{:.5f}".format(time.time() - t))

/tmp/ipykernel_28338/3789869377.py:98: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  adj = adj.astype(np.float)
/tmp/ipykernel_28338/3789869377.py:147: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  adj_label = adj_label.astype(np.float)
/tmp/ipykernel_28338/3789869377.py:171: DeprecationWarning: `np.float` is a deprecated alias for the bu

TypeError: Module.children() missing 1 required positional argument: 'self'